In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import cv2 as cv2
import numpy as np

import torch
import torch.hub

import torchvision
import torchvision.utils

from data import dataset as data
import data.utils as data_utils
import attack_construction.attack_methods as attack_methods
import attack_construction.metrics as metrics
import attack_construction.utils as attack_utils

print(torch.__version__)
print(torch.cuda_version)
print(torchvision.__version__)

1.10.0+cu113
11.3
0.11.1+cu113


In [2]:
device = torch.device("cpu")

#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
model.eval()
model = model.float().to(device)

dataset = data.AdversarialDataset() # todo: use resize to pull picture in batch

In [3]:
patch = attack_utils.load_tensor("patch - new")
device = torch.device("cpu")

In [14]:
image, label = dataset[5]

patch = torch.rand(3, 1000, 4)

image_before, image_after = attack_utils.get_patch_test(patch, image, label, model, device, None, 0.5)

cv2.imshow("patch", cv2.cvtColor(np.concatenate((image_before/255, image_after/255), axis = 1), cv2.COLOR_RGB2BGR))

key = cv2.waitKey(0)
if key & 0xFF == ord('q'):
    cv2.destroyAllWindows()

In [5]:
image, label = dataset[5]
pred_label = model([data_utils.image_to_tensor(image)])[0]


C:\Users\Степан\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [14]:
image, label, _ = dataset[5]
print(image.shape)
print

(3, 300, 800)


In [18]:
print(label)

tensor([[103.3200, 133.4500, 231.0300, 421.8900],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000]], dtype=torch.float64)


In [41]:
print(pred_label['scores'])

tensor([0.9512, 0.8752, 0.8034, 0.7582, 0.7398, 0.6379, 0.6187, 0.5611, 0.5536,
        0.5497, 0.4910, 0.4495, 0.4459, 0.4458, 0.4436, 0.4421, 0.3846, 0.3770,
        0.3603, 0.3546, 0.3243, 0.2736, 0.2644, 0.2584, 0.2577, 0.2550, 0.2480,
        0.2450, 0.2309, 0.2288, 0.2191, 0.2105, 0.2077, 0.1965, 0.1937, 0.1935,
        0.1928, 0.1863, 0.1857, 0.1797, 0.1781, 0.1781, 0.1779, 0.1734, 0.1702,
        0.1660, 0.1554, 0.1507, 0.1502, 0.1493, 0.1489, 0.1482, 0.1481, 0.1467,
        0.1444, 0.1427, 0.1423, 0.1410, 0.1408, 0.1390, 0.1367, 0.1342, 0.1342,
        0.1340, 0.1337, 0.1325, 0.1320, 0.1291, 0.1286, 0.1282, 0.1281, 0.1270,
        0.1264, 0.1264, 0.1263, 0.1261, 0.1259, 0.1223, 0.1211, 0.1208, 0.1208,
        0.1201, 0.1183, 0.1178, 0.1164, 0.1159, 0.1157, 0.1149, 0.1141, 0.1134,
        0.1133, 0.1127, 0.1125, 0.1123, 0.1117, 0.1103, 0.1099, 0.1094, 0.1078,
        0.1067, 0.1067, 0.1054, 0.1053, 0.1051, 0.1045, 0.1043, 0.1031, 0.1029,
        0.1028, 0.1019, 0.1016, 0.1015, 

In [38]:
metrics.intersection_over_union(pred_label['boxes'][0].detach(), label[0].detach())

tensor([0.9078], dtype=torch.float64)

In [4]:
device = torch.device("cuda:0")
dataset = data.AdversarialDataset((800,300))
dataset_val = data.AdversarialDataset((640,640), '../../val2017/val2017', '../../annotations_trainval2017/annotations/instances_val2017.json')
train_loader  = torch.utils.data.DataLoader(dataset=dataset, batch_size=3, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, 
batch_size=3, shuffle=False)
def loss_function(predict, patch, device):
        return metrics.general_objectness(predict, device) + 0 * metrics.total_variation(patch)


In [65]:
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
model.eval()
model = model.float().to(device)

In [55]:
print(dataset_val[0])

(array([[[124, 128,  77],
        [120, 126,  64],
        [104, 111,  57],
        ...,
        [ 11,  12,  14],
        [  9,  10,  12],
        [ 12,  13,  15]],

       [[119, 129,  72],
        [120, 131,  68],
        [ 99, 110,  54],
        ...,
        [ 12,  12,  13],
        [  9,  10,  11],
        [ 11,  12,  12]],

       [[115, 130,  68],
        [119, 135,  72],
        [ 93, 108,  51],
        ...,
        [ 12,  12,  12],
        [ 15,  15,  15],
        [ 19,  19,  17]],

       ...,

       [[140, 105,  73],
        [143, 108,  76],
        [140, 107,  76],
        ...,
        [ 48,  31,  23],
        [ 50,  31,  24],
        [ 46,  32,  23]],

       [[141, 106,  72],
        [143, 109,  75],
        [141, 108,  75],
        ...,
        [ 47,  30,  21],
        [ 50,  31,  23],
        [ 47,  32,  22]],

       [[142, 108,  71],
        [144, 110,  75],
        [142, 109,  74],
        ...,
        [ 47,  29,  19],
        [ 50,  32,  22],
        [ 49,  32,  22]

In [67]:
patch = patch.to(device)
loss_before, loss_after, mAP = attack_methods.validate(model, patch, None, val_loader, loss_function, device)

..\data\utils.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image).permute(2, 0, 1).float() / 255


KeyboardInterrupt: 

In [69]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

cocoGt=COCO("../../annotations_trainval2017/annotations/instances_val2017.json")
cocoDt=cocoGt.loadRes("./tmp.json")

cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
cocoEval.params.imgIds = cocoDt.getImgIds()
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=1.41s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=13.78s).
Accumulating evaluation results...
DONE (t=2.49s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

In [63]:
cocoDt.getImgIds()

[397133,
 37777,
 252219,
 87038,
 174482,
 403385,
 6818,
 480985,
 458054,
 331352,
 296649,
 386912,
 502136,
 491497,
 184791,
 348881,
 289393,
 522713,
 181666,
 17627,
 143931,
 303818,
 463730,
 460347,
 322864,
 226111,
 153299,
 308394,
 456496,
 58636,
 41888,
 184321,
 565778,
 297343,
 336587,
 122745,
 219578,
 555705,
 443303,
 500663,
 418281,
 25560,
 403817,
 85329,
 329323,
 239274,
 286994,
 511321,
 314294,
 233771,
 475779,
 301867,
 312421,
 185250,
 356427,
 572517,
 270244,
 516316,
 125211,
 562121,
 360661,
 16228,
 382088,
 266409,
 430961,
 80671,
 577539,
 104612,
 476258,
 448365,
 35197,
 349860,
 180135,
 486438,
 400573,
 109798,
 370677,
 238866,
 369370,
 502737,
 515579,
 515445,
 173383,
 438862,
 180560,
 347693,
 39956,
 321214,
 474028,
 66523,
 355257,
 142092,
 63154,
 199551,
 239347,
 514508,
 473237,
 228144,
 206027,
 78915,
 551215,
 544519,
 96493,
 23899,
 340175,
 578500,
 366141,
 57597,
 559842,
 434230,
 428454,
 399462,
 261061,
 1

In [71]:
print(a)

None


In [73]:
np.mean(cocoEval.stats)

0.0

In [12]:
patch = attack_methods.generate_random_patch(resolution=(70, 70))
device = torch.device("cpu")

for images, labels, img_ids in val_loader:
        attacked_images = [i for i in images]
        images = images.to(device)

        print('images in batch: ', len(images))

        for i, immage in enumerate(images):
            print("labels in image[", i,"]: ",len(labels[i]))
            for label in labels[i]:
                attacked_images.append(attack_methods.insert_patch(attacked_images[i], patch, label, 0.3, device))

        print("attacked size: ",len(attacked_images))
        break
    

images in batch:  3
labels in image[ 0 ]:  10
labels in image[ 1 ]:  10
labels in image[ 2 ]:  10
attacked size:  33
